# Import Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf

import os
from zipfile import ZipFile

In [2]:
%matplotlib inline

# Download Data

In [3]:
import wget
wget.download("https://github.com/alexeygrigorev/dino-or-dragon/releases/download/data/dino-dragon.zip")

100% [......................................................................] 109298562 / 109298562

'dino-dragon.zip'

In [4]:
with ZipFile("dino-dragon.zip","r") as zip_obj:
    zip_obj.extractall()

The dataset contains around 1900 images of dinos and around 1900 images of dragons.

The dataset contains separate folders for training and validation.
Model

In [5]:
train_path = "./train/"
validation_path = "./test/"

For this homework we will use Convolutional Neural Network (CNN). Like in the lectures, we'll use Keras.

You need to develop the model with following structure:

    The shape for input should be (150, 150, 3)
    Next, create a convolutional layer (Conv2D):
        Use 32 filters
        Kernel size should be (3, 3) (that's the size of the filter)
        Use 'relu' as activation
    Reduce the size of the feature map with max pooling (MaxPooling2D)
        Set the pooling size to (2, 2)
    Turn the multi-dimensional result into vectors using a Flatten layer
    Next, add a Dense layer with 64 neurons and 'relu' activation
    Finally, create the Dense layer with 1 neuron - this will be the output
        The output layer should have an activation - use the appropriate activation for the binary classification case

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.activations import relu, sigmoid

In [7]:
# Model Parameters
input_shape = (150, 150, 3)
filters= 32
kernel_size = (3, 3)
pool_size = (2, 2)
conv2d_activation = relu
dense_layer_activation = relu
output_activation = sigmoid
dense_layer_1_neurons = 64
output_layer_neurons = 1

In [8]:
# Model Definition
model = Sequential([
                    InputLayer(input_shape=input_shape),
                    Conv2D(filters=filters, kernel_size=kernel_size, 
                           kernel_initializer='glorot_normal', activation=conv2d_activation),
                    MaxPooling2D(pool_size=pool_size),
                    Flatten(),
                    Dense(units=dense_layer_1_neurons, activation=dense_layer_activation),
                    Dense(units=output_layer_neurons, activation=output_activation)
                  ])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 175232)            0         
                                                                 
 dense (Dense)               (None, 64)                11214912  
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 11,215,873
Trainable params: 11,215,873
Non-trainable params: 0
____________________________________________

As optimizer use SGD with the following parameters:

    SGD(lr=0.002, momentum=0.8)

For clarification about kernel size and max pooling, check Office Hours.

In [9]:
# Import Model Optimizer
from tensorflow.keras.optimizers import SGD  

In [10]:
# Optimizer Parameters
learning_rate = 0.002
momentum = 0.8

In [11]:
# Optimizer Definition
optimizer = SGD(learning_rate=learning_rate,
                momentum=momentum, 
                name='SGD')

In [12]:
# Loss Definition
loss = 'binary_crossentropy' 

In [13]:
# Metrics definition
metrics = ['accuracy']

In [14]:
# Compile Model
model.compile(optimizer=optimizer,
              loss=loss, 
              metrics=metrics) 

## Question 1

Since we have a binary classification problem, what is the best loss function for us?

- **binary crossentropy**
- focal loss
- mean squared error
- categorical crossentropy
    
Note: since we specify an activation for the output layer, we don't need to set from_logits=True


## Question 2

What's the total number of parameters of the model? You can use the summary method for that.

- 9215873
- **11215873**
- 14215873
- 19215873

# Generators and Training

For the next two questions, use the following data generator for both train and validation:

ImageDataGenerator(rescale=1./255)

    We don't need to do any additional pre-processing for the images.
    
    When reading the data from train/val directories, check the class_mode parameter. Which value should it be for a binary classification problem?
    
    

***class_mode should be binary in this case***

Use batch_size=20

Use shuffle=True for both training and validation

In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [16]:
target_size=(150, 150)
color_mode="rgb"
class_mode='binary'
batch_size=20
shuffle=True 
seed=100

In [17]:
train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(train_path,
                                                                        target_size=target_size,
                                                                        color_mode=color_mode,
                                                                        classes=os.listdir(train_path),
                                                                        class_mode=class_mode,
                                                                        batch_size=batch_size,
                                                                        shuffle=shuffle,
                                                                        seed=seed
                                                                        )


Found 1594 images belonging to 2 classes.


In [18]:
validation_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(validation_path,
                                                                             target_size=target_size,
                                                                             color_mode=color_mode,
                                                                             classes=os.listdir(validation_path),
                                                                             class_mode=class_mode,
                                                                             batch_size=batch_size,
                                                                             shuffle=shuffle,
                                                                             seed=seed
                                                                             )

Found 394 images belonging to 2 classes.


For training use .fit() with the following params:

model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

In [19]:
# Training the Data
history = model.fit(train_generator, 
                    batch_size=20,
                    epochs=10, 
                    validation_data=validation_generator,
                    validation_batch_size=20,
                    shuffle=True
                   )

Epoch 1/10
80/80 [==============================] - 16s 194ms/step - loss: 0.6443 - accuracy: 0.6198 - val_loss: 0.6257 - val_accuracy: 0.5964
Epoch 2/10
80/80 [==============================] - 15s 189ms/step - loss: 0.5111 - accuracy: 0.7610 - val_loss: 0.4639 - val_accuracy: 0.8147
Epoch 3/10
80/80 [==============================] - 16s 195ms/step - loss: 0.4334 - accuracy: 0.8193 - val_loss: 0.4274 - val_accuracy: 0.8046
Epoch 4/10
80/80 [==============================] - 15s 189ms/step - loss: 0.3838 - accuracy: 0.8369 - val_loss: 0.4016 - val_accuracy: 0.7995
Epoch 5/10
80/80 [==============================] - 15s 190ms/step - loss: 0.3390 - accuracy: 0.8657 - val_loss: 0.4612 - val_accuracy: 0.7817
Epoch 6/10
80/80 [==============================] - 16s 195ms/step - loss: 0.3044 - accuracy: 0.8833 - val_loss: 0.3330 - val_accuracy: 0.8503
Epoch 7/10
80/80 [==============================] - 15s 185ms/step - loss: 0.2797 - accuracy: 0.8934 - val_loss: 0.3133 - val_accuracy: 0.8604

## Question 3

What is the median of training accuracy for all the epochs for this model?

- 0.40
- 0.60
- **0.90**
- 0.20

In [20]:
accuracy = history.history.get("accuracy")
accuracy.sort()
np.median(accuracy)

0.8745294809341431

## Question 4

What is the standard deviation of training loss for all the epochs for this model?

- **0.11**
- 0.66
- 0.99
- 0.33

In [21]:
training_loss = history.history.get("loss")
np.array(training_loss).std()

0.1333373644863049

# Data Augmentation

For the next two questions, we'll generate more data using data augmentations.

Add the following augmentations to your training data generator:

    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'

In [22]:
batch_size=20
shuffle=True 
target_size=(150, 150)
seed = 100
color_mode="rgb"
class_mode='binary'

In [23]:
train_generator = ImageDataGenerator(rescale=1./255,
                                     rotation_range=40,
                                     width_shift_range=0.2,
                                     height_shift_range=0.2,
                                     shear_range=0.2,
                                     zoom_range=0.2,
                                     horizontal_flip=True,
                                     fill_mode='nearest'
                                    ).flow_from_directory(train_path,
                                                          target_size=target_size,
                                                          color_mode=color_mode,
                                                          classes=None, # or os.listdir(train_path)
                                                          class_mode=class_mode,
                                                          batch_size=batch_size,
                                                          shuffle=shuffle,
                                                          seed=seed
                                                         )


Found 1594 images belonging to 2 classes.


In [24]:
validation_generator = ImageDataGenerator(rescale=1./255,
                                          rotation_range=40,
                                          width_shift_range=0.2,
                                          height_shift_range=0.2,
                                          shear_range=0.2,
                                          zoom_range=0.2,
                                          horizontal_flip=True,
                                          fill_mode='nearest'
                                         ).flow_from_directory(validation_path,
                                                               target_size=target_size,
                                                               color_mode=color_mode,
                                                               classes=None, # or os.listdir(train_path)
                                                               class_mode=class_mode,
                                                               batch_size=batch_size,
                                                               shuffle=shuffle,
                                                               seed=seed
                                                              )

Found 394 images belonging to 2 classes.


## Question 5

Let's train our model for 10 more epochs using the same code as previously. Make sure you don't re-create the model - we want to continue training the model we already started training.

What is the mean of validation loss for all the epochs for the model trained with augmentations?

- 0.15
- 0.77
- **0.37**
- 0.97

In [25]:
# Training the Data
history = model.fit(train_generator, 
                    batch_size=20,
                    epochs=10, 
                    validation_data=validation_generator,
                    validation_batch_size=10,
                    shuffle=True
                   )

Epoch 1/10
80/80 [==============================] - 33s 412ms/step - loss: 0.4493 - accuracy: 0.7905 - val_loss: 0.4984 - val_accuracy: 0.7462
Epoch 2/10
80/80 [==============================] - 33s 411ms/step - loss: 0.4291 - accuracy: 0.7986 - val_loss: 0.3667 - val_accuracy: 0.8452
Epoch 3/10
80/80 [==============================] - 33s 412ms/step - loss: 0.4303 - accuracy: 0.7986 - val_loss: 0.4114 - val_accuracy: 0.7893
Epoch 4/10
80/80 [==============================] - 33s 414ms/step - loss: 0.4133 - accuracy: 0.8137 - val_loss: 0.3757 - val_accuracy: 0.8376
Epoch 5/10
80/80 [==============================] - 33s 414ms/step - loss: 0.3807 - accuracy: 0.8306 - val_loss: 0.3567 - val_accuracy: 0.8477
Epoch 6/10
80/80 [==============================] - 33s 413ms/step - loss: 0.3839 - accuracy: 0.8306 - val_loss: 0.3918 - val_accuracy: 0.8299
Epoch 7/10
80/80 [==============================] - 33s 415ms/step - loss: 0.3695 - accuracy: 0.8444 - val_loss: 0.4147 - val_accuracy: 0.7893

In [26]:
validation_loss = history.history.get("val_loss") 
np.array(validation_loss).mean()

0.38821635246276853

## Question 6

What's the average of validation accuracy for the last 5 epochs (from 6 to 10) for the model trained with augmentations?

- **0.84**
- 0.54
- 0.44
- 0.24

In [27]:
validation_accuracy = history.history.get("val_accuracy")
np.array(validation_accuracy[-5:]).mean()

0.8335025310516357